In [1]:
from ml.data.datasets import StarWars

dataset = StarWars()
data = dataset[0]
data

[2022-02-12 13:51:37,934][simple_parsing.helpers.serialization.serializable][DEBUG] Registering a new Serializable subclass: <class 'simple_parsing.helpers.serialization.serializable.Serializable'>
[2022-02-12 13:51:37,935][simple_parsing.helpers.serialization.serializable][DEBUG] parents: [<class 'simple_parsing.helpers.serialization.serializable.SerializableMixin'>]
[2022-02-12 13:51:37,936][simple_parsing.helpers.serialization.serializable][DEBUG] Registering a new Serializable subclass: <class 'simple_parsing.helpers.serialization.serializable.FrozenSerializable'>
[2022-02-12 13:51:37,937][simple_parsing.helpers.serialization.serializable][DEBUG] parents: [<class 'simple_parsing.helpers.serialization.serializable.SerializableMixin'>]
[2022-02-12 13:51:37,938][simple_parsing.helpers.serialization.serializable][DEBUG] Registering a new Serializable subclass: <class 'simple_parsing.helpers.serialization.serializable.SimpleSerializable'>
[2022-02-12 13:51:37,941][simple_parsing.helpers

HeteroData(
  Character={ x=[113, 32] },
  (Character, INTERACTIONS, Character)={
    edge_attr=[958, 0],
    edge_index=[2, 958],
    timestamp=[958]
  },
  (Character, MENTIONS, Character)={
    edge_attr=[1120, 0],
    edge_index=[2, 1120],
    timestamp=[1120]
  }
)

In [28]:
from ml.data import EdgeLoaderDataModule

data_module = EdgeLoaderDataModule(data, batch_size=16, num_samples=[4] * 2, num_workers=4, node_type='Character')

In [29]:
from typing import Optional, Dict, Tuple, Any

import torch
import torch.nn.functional as F
from torch_geometric.typing import Metadata
from torch_geometric.nn import HGTConv, Linear
import torchmetrics
import pytorch_lightning as pl
import math

import ml

In [30]:
class HGTModule(torch.nn.Module):
    def __init__(
            self,
            node_type,
            metadata: Metadata,
            hidden_channels=64,
            num_heads=2,
            num_layers=1
    ):
        super().__init__()
        self.node_type = node_type
        self.lin_dict = torch.nn.ModuleDict()
        for node_type in metadata[0]:
            self.lin_dict[node_type] = Linear(-1, hidden_channels)

        self.convs = torch.nn.ModuleList()
        for _ in range(num_layers):
            conv = HGTConv(hidden_channels, hidden_channels, metadata, num_heads, group='sum')
            self.convs.append(conv)

    def forward(self, x_dict, edge_index_dict):
        for node_type, x in x_dict.items():
            x_dict[node_type] = self.lin_dict[node_type](x).relu_()

        for conv in self.convs:
            x_dict = conv(x_dict, edge_index_dict)

        return x_dict[self.node_type]

class ClusteringModule(torch.nn.Module):
    def __init__(self, rep_dim, n_clusters):
        super().__init__()
        self.lin = torch.nn.Linear(rep_dim, n_clusters)
        self.activation = torch.nn.Softmax(dim=1)

    def forward(self, batch: torch.Tensor):
        return self.activation(self.lin(batch))

    def forward_assign(self, batch: torch.Tensor):
        q = self(batch)
        return q.argmax(dim=1)

class LinkPredictionLoss(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.ce_loss = torch.nn.CrossEntropyLoss()

    def forward(self, pred: torch.Tensor, label: torch.Tensor):
        return self.ce_loss(pred, label)

class ClusteringLoss(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.cos_sim = torch.nn.CosineSimilarity(dim=1)
        self.ce_loss = torch.nn.CrossEntropyLoss()

    def forward(self, q_l: torch.Tensor, q_r: torch.Tensor, label: torch.Tensor):
        p = torch.sum(q_l, dim=0) + torch.sum(q_r, dim=0)
        p /= p.sum()
        ne = math.log(p.size(0)) + (p * torch.log(p)).sum()

        sim = (self.cos_sim(q_l, q_r) + 1) / 2
        pred = torch.stack([1 - sim, sim], dim=1)
        loss = self.ce_loss(pred, label)

        return loss, ne

In [50]:
class Net(ml.BaseModule):
    def __init__(
            self,
            embedding_module: HGTModule,
            clustering_module: ClusteringModule,
    ):
        super().__init__()
        self.embedding_module = embedding_module
        self.clustering_module = clustering_module
        self.cos_sim = torch.nn.CosineSimilarity(dim=1)
        self.lin = torch.nn.Linear(1, 2)

        self.link_prediction_loss = LinkPredictionLoss()
        self.clustering_loss = ClusteringLoss()

    def configure_metrics(self) -> Dict[str, Tuple[torchmetrics.Metric, bool]]:
        return {
            'loss': (torchmetrics.MeanMetric(), True),
            'hp_loss': (torchmetrics.MeanMetric(), True),
            'cc_loss': (torchmetrics.MeanMetric(), True),
            'accuracy': (torchmetrics.Accuracy(), True),
            'ne': (torchmetrics.MeanMetric(), True),
        }

    def _step(self, batch: torch.Tensor):
        batch_l, batch_r, label = batch
        batch_size = batch_l[self.embedding_module.node_type].batch_size

        emb_l = self.embedding_module(batch_l.x_dict, batch_l.edge_index_dict)[:batch_size]
        emb_r = self.embedding_module(batch_r.x_dict, batch_r.edge_index_dict)[:batch_size]
        sim = self.cos_sim(emb_l, emb_r)
        out = self.lin(torch.unsqueeze(sim, 1))
        hp_loss = self.link_prediction_loss(out, label)

        q_l = self.clustering_module(emb_l)
        q_r = self.clustering_module(emb_r)
        cc_loss, ne = self.clustering_loss(q_l, q_r, label)

        loss = hp_loss + cc_loss + ne * 0.1

        pred = out.argmax(dim=-1)
        return {
            'loss': loss,
            'hp_loss': hp_loss,
            'cc_loss': cc_loss,
            'accuracy': (pred, label),
            'ne': ne,
        }

    def forward(self, batch):
        batch_size = batch[self.embedding_module.node_type].batch_size
        emb = self.embedding_module(batch.x_dict, batch.edge_index_dict)[:batch_size]
        q = self.clustering_module(emb)
        return emb, q

    def training_step(self, batch):
        return self._step(batch)

    def validation_step(self, batch, batch_idx):
        return self._step(batch)

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
        return optimizer

In [51]:
embedding_module = HGTModule(node_type='Character', metadata=data.metadata(), hidden_channels=32, num_heads=2, num_layers=2)
clustering_module = ClusteringModule(rep_dim=32, n_clusters=5)

In [52]:
model = Net(embedding_module, clustering_module)

In [53]:
trainer = pl.Trainer(
    # gpus=1,
    callbacks=[
        pl.callbacks.EarlyStopping(monitor="val/loss", min_delta=0.00, patience=5, verbose=True, mode="min")
    ],
    max_epochs=50,
    enable_model_summary=True,
    # logger=wandb_logger
)
trainer.fit(model, data_module)

[2022-02-12 14:58:27,145][py.warnings][WARNING] /data/pella/projects/University/Thesis/Thesis/code/env/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py:1584: UserWarning: GPU available but not used. Set the gpus flag in your trainer `Trainer(gpus=1)` or script `--gpus=1`.
  rank_zero_warn(

[2022-02-12 14:58:27,148][py.warnings][WARNING] /data/pella/projects/University/Thesis/Thesis/code/env/lib/python3.9/site-packages/pytorch_lightning/core/datamodule.py:469: LightningDeprecationWarning: DataModule.setup has already been called, so it will not be called again. In v1.6 this behavior will change to always call DataModule.setup.
  rank_zero_deprecation(



Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

[2022-02-12 14:58:27,471][py.warnings][WARNING] /data/pella/projects/University/Thesis/Thesis/code/env/lib/python3.9/site-packages/pytorch_lightning/loops/optimization/closure.py:35: LightningDeprecationWarning: One of the returned values {'cc_loss', 'ne', 'hp_loss', 'accuracy'} has a `grad_fn`. We will detach it automatically but this behaviour will change in v1.6. Please detach it manually: `return {'loss': ..., 'something': something.detach()}`
  rank_zero_deprecation(



Validating: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

[2022-02-12 15:04:05,868][py.warnings][WARNING] /data/pella/projects/University/Thesis/Thesis/code/env/lib/python3.9/site-packages/pytorch_lightning/core/datamodule.py:469: LightningDeprecationWarning: DataModule.teardown has already been called, so it will not be called again. In v1.6 this behavior will change to always call DataModule.teardown.
  rank_zero_deprecation(



In [54]:
from shared.constants import BENCHMARKS_RESULTS

save_path = BENCHMARKS_RESULTS.joinpath('analysis', 'pyg-hgt-comopt-contrastive')
save_path.mkdir(parents=True, exist_ok=True)

In [55]:
predictions = trainer.predict(model, data_module)
embeddings, assignments = map(lambda x: torch.cat(x, dim=0).detach().cpu(), zip(*predictions))
assignments = torch.argmax(assignments, dim=1)

Predicting: 182it [00:00, ?it/s]

In [56]:
from shared.graph import CommunityAssignment
import pandas as pd

labeling = pd.Series(assignments.squeeze(), index=dataset.node_mapping(), name="cid")
labeling.index.name = "nid"
comlist = CommunityAssignment(labeling)

In [57]:
comlist.save_comlist(save_path.joinpath('schema.comlist'))

In [ ]:
from datasets.scripts import export_to_visualization

export_to_visualization.run(
    export_to_visualization.Args(
        dataset='star-wars',
        version='base',
        run_paths=[str(save_path)]
    )
)

[2022-02-12 15:04:06,132][simple_parsing.helpers.serialization.serializable][DEBUG] from_dict for <class 'shared.schema.dataset.DatasetSchema'>, drop extra fields: True
[2022-02-12 15:04:06,132][simple_parsing.helpers.serialization.decoding][DEBUG] name = name, field_type = <class 'str'>
[2022-02-12 15:04:06,133][simple_parsing.helpers.serialization.decoding][DEBUG] name = database, field_type = <class 'str'>
[2022-02-12 15:04:06,134][simple_parsing.helpers.serialization.decoding][DEBUG] name = description, field_type = <class 'str'>
[2022-02-12 15:04:06,134][simple_parsing.helpers.serialization.decoding][DEBUG] name = versions, field_type = typing.Dict[str, shared.schema.dataset.DatasetVersion]
[2022-02-12 15:04:06,135][simple_parsing.helpers.serialization.serializable][DEBUG] from_dict for <class 'shared.schema.dataset.DatasetVersion'>, drop extra fields: True
[2022-02-12 15:04:06,136][simple_parsing.helpers.serialization.decoding][DEBUG] name = type, field_type = <enum 'DatasetVersi

# Calculate Evaluation Metrics

In [58]:
from shared.graph import DataGraph
from benchmarks.evaluation import get_metric_list

In [59]:
from shared.schema import GraphSchema, DatasetSchema

DATASET = DatasetSchema.load_schema('star-wars')
schema = GraphSchema.from_dataset(DATASET)
G = DataGraph.from_schema(schema)

ERROR! Session/line number was not unique in database. History logging moved to new session 164
[2022-02-12 15:04:06,768][simple_parsing.helpers.serialization.serializable][DEBUG] from_dict for <class 'shared.schema.dataset.DatasetSchema'>, drop extra fields: True
[2022-02-12 15:04:06,769][simple_parsing.helpers.serialization.decoding][DEBUG] name = name, field_type = <class 'str'>
[2022-02-12 15:04:06,770][simple_parsing.helpers.serialization.decoding][DEBUG] name = database, field_type = <class 'str'>
[2022-02-12 15:04:06,770][simple_parsing.helpers.serialization.decoding][DEBUG] name = description, field_type = <class 'str'>
[2022-02-12 15:04:06,771][simple_parsing.helpers.serialization.decoding][DEBUG] name = versions, field_type = typing.Dict[str, shared.schema.dataset.DatasetVersion]
[2022-02-12 15:04:06,772][simple_parsing.helpers.serialization.serializable][DEBUG] from_dict for <class 'shared.schema.dataset.DatasetVersion'>, drop extra fields: True
[2022-02-12 15:04:06,772][sim

In [60]:
metrics = get_metric_list(ground_truth=False, overlapping=False)

results = pd.DataFrame([
    {
        'metric': metric_cls.metric_name(),
        'value': metric_cls.calculate(G, comlist)
    }
    for metric_cls in metrics]
)
results

,metric,value
0,community_count,NaN
1,conductance,0.467894
2,expansion,4.787290
3,internal_edge_density,0.331168
4,avg_odf,9.706644
5,modularity_overlap,0.028058
6,link_modularity,0.038189
7,z_modularity,0.396090
8,modularity,-0.076791
